# Introduction

![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

# Tasks

The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called `ds_jobs_transformed` that stores the data in `customer_train.csv` much more efficiently. Specifically, they have set the following requirements:

- [x] Columns containing categories with only two factors must be stored as Booleans (`bool`).
- [x] Columns containing integers *only* must be stored as 32-bit integers (`int32`).
- [x] Columns containing floats must be stored as 16-bit floats (`float16`).
- [x] Columns containing nominal categorical data must be stored as the `category` data type.
- [x] Columns containing ordinal categorical data must be stored as *ordered categories*, and not mapped to numerical values, with an order that reflects the natural order of the column.
- [x] The DataFrame should be filtered to only contain students with *10 or more years of experience* at companies with *at least 1000 employees*, as their recruiter base is suited to more experienced professionals at enterprise companies.

If you call `.info()` or `.memory_usage()` methods on `ds_jobs` and `ds_jobs_transformed` after you've preprocessed it, you should notice a substantial decrease in memory usage.

# Importing and a first look

In [1]:
import pandas as pd
import numpy as np

In [2]:
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs_transformed = ds_jobs.copy()

In [3]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [4]:
ds_jobs.nunique()

student_id                19158
city                        123
city_development_index       93
gender                        3
relevant_experience           2
enrolled_university           3
education_level               5
major_discipline              6
experience                   22
company_size                  8
company_type                  6
last_new_job                  6
training_hours              241
job_change                    2
dtype: int64

# Boolean coding

First task: 

- Columns containing categories with only two factors must be stored as Booleans (`bool`).

## Showing the columns with only two factors

In [5]:
ds_jobs.nunique()[
    ds_jobs.nunique() == 2
]

relevant_experience    2
job_change             2
dtype: int64

In [6]:
ds_jobs['relevant_experience'].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [7]:
ds_jobs['job_change'].unique()

array([1., 0.])

## Changing the columns

In [8]:
ds_jobs_transformed['relevant_experience'] = np.where(
    ds_jobs['relevant_experience'] == 'Has relevant experience',
    True, False)

In [9]:
ds_jobs_transformed['relevant_experience'].dtype

dtype('bool')

In [10]:
ds_jobs_transformed['job_change'] = np.where(
    ds_jobs['job_change'] == 1,
    True, False
)

In [11]:
ds_jobs_transformed['job_change'].dtype

dtype('bool')

# int and float columns

Next tasks:
- Columns containing integers *only* must be stored as 32-bit integers (`int32`).
- Columns containing floats must be stored as 16-bit floats (`float16`).

In [12]:
ds_jobs_transformed.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                   bool
dtype: object

In [13]:
int_cols = ['student_id', 'training_hours']

for col in int_cols:
    ds_jobs_transformed[col] = ds_jobs[col].astype('int32')

In [14]:
float_cols = ['city_development_index']

for col in float_cols:
    ds_jobs_transformed[col] = ds_jobs[col].astype('float16')

# Categorical columns

Next tasks:
- Columns containing nominal categorical data must be stored as the `category` data type.
- Columns containing ordinal categorical data must be stored as *ordered categories*, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [15]:
ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

## Nominal categorical data

In [16]:
nom_cat_cols = [
    'city', 
    'gender', 
    'major_discipline', 
    'company_type'
]

In [17]:
for col in nom_cat_cols:
    ds_jobs_transformed[col] = ds_jobs[col].astype('category')

In [19]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  object  
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  object  
 9   company_size            13220 non-null  object  
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  object  
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(5

## Ordinal categorical data

### Enrolled in university

In [33]:
ds_jobs['enrolled_university'].value_counts()

enrolled_university
no_enrollment       13817
Full time course     3757
Part time course     1198
Name: count, dtype: int64

In [34]:
enrolled_university_vals = ['no_enrollment', 'Part time course', 'Full time course']

ds_jobs_transformed['enrolled_university'] = ds_jobs['enrolled_university'].astype('category')

ds_jobs_transformed['enrolled_university'] =\
    ds_jobs_transformed['enrolled_university'].cat.reorder_categories(
        new_categories = enrolled_university_vals,
        ordered=True
    )

### Education level

In [20]:
ds_jobs['education_level'].value_counts()

education_level
Graduate          11598
Masters            4361
High School        2017
Phd                 414
Primary School      308
Name: count, dtype: int64

In [21]:
education_levels = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']

ds_jobs_transformed['education_level'] = ds_jobs['education_level'].astype('category')

ds_jobs_transformed[
    'education_level'
] = ds_jobs_transformed[
    'education_level'
].cat.reorder_categories(
    new_categories = education_levels,
    ordered=True
)

### Experience

In [22]:
ds_jobs['experience'].value_counts()

experience
>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: count, dtype: int64

In [23]:
experience_levels = ['<1'] +\
    [str(n) for n in range(1,21)] +\
    ['>20']

ds_jobs_transformed['experience'] = ds_jobs['experience'].astype('category')

ds_jobs_transformed[
    'experience'
] = ds_jobs_transformed[
    'experience'
].cat.reorder_categories(
    new_categories = experience_levels,
    ordered=True
)

In [24]:
ds_jobs_transformed['experience']

0        >20
1         15
2          5
3         <1
4        >20
        ... 
19153     14
19154     14
19155    >20
19156     <1
19157      2
Name: experience, Length: 19158, dtype: category
Categories (22, object): ['<1' < '1' < '2' < '3' ... '18' < '19' < '20' < '>20']

### Company size

In [25]:
ds_jobs['company_size'].value_counts()

company_size
50-99        3083
100-499      2571
10000+       2019
10-49        1471
1000-4999    1328
<10          1308
500-999       877
5000-9999     563
Name: count, dtype: int64

In [26]:
company_sizes = [
    '<10',
    '10-49',
    '50-99',
    '100-499',
    '500-999',
    '1000-4999',
    '5000-9999',
    '10000+'
]

ds_jobs_transformed['company_size'] = ds_jobs['company_size'].astype('category')

ds_jobs_transformed[
    'company_size'
] = ds_jobs_transformed[
    'company_size'
].cat.reorder_categories(new_categories = company_sizes, ordered=True)

ds_jobs_transformed['company_size']

0            NaN
1          50-99
2            NaN
3            NaN
4          50-99
          ...   
19153        NaN
19154        NaN
19155      50-99
19156    500-999
19157        NaN
Name: company_size, Length: 19158, dtype: category
Categories (8, object): ['<10' < '10-49' < '50-99' < '100-499' < '500-999' < '1000-4999' < '5000-9999' < '10000+']

### Last new job

In [27]:
ds_jobs['last_new_job'].value_counts()

last_new_job
1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: count, dtype: int64

In [28]:
last_new_job_durations = ['never'] + [str(n) for n in range(1,5)] + ['>4']

ds_jobs_transformed['last_new_job'] = ds_jobs['last_new_job'].astype('category')

ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(
    new_categories = last_new_job_durations,
    ordered=True
)

In [29]:
ds_jobs_transformed['last_new_job']

0            1
1           >4
2        never
3        never
4            4
         ...  
19153        1
19154        4
19155        4
19156        2
19157        1
Name: last_new_job, Length: 19158, dtype: category
Categories (6, object): ['never' < '1' < '2' < '3' < '4' < '>4']

# Filtering

Last task:
- The DataFrame should be filtered to only contain students with *10 or more years of experience* at companies with *at least 1000 employees*, as their recruiter base is suited to more experienced professionals at enterprise companies.


In [30]:
ds_jobs_transformed = ds_jobs_transformed[
    (ds_jobs_transformed['experience'] >= '10')
    &
    (ds_jobs_transformed['company_size'] >= '1000-4999')
].copy()

# Memory savings?

In [31]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [32]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   bool    
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              2201 non-null   category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   bool    
dtypes: bool(2), category(9), flo